In [1]:
import numpy as np

from scipy.sparse import csr_matrix, csc_matrix
from scipy.sparse.csgraph import csgraph_from_dense

# Типы и Структуры Данных. ЛР4
## Разреженные матрицы

Необходимо реализовать либо КРМ, либо РСФ: упаковку, распаковку, сложение двух матриц и произведение двух матриц.

В лабораторной не допускается сплошное извлечение элементов матриц (вида "извлечь элемент с координатами [i][j] из упакованной матрицы"), требуется использовать алгоритмы просмотра массива ненулевых элементов.

In [2]:
A = np.array([
    [5, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [1, 0, 4, 0, 0, 4, 0, 0, 0],
    [1, 0, 3, 0, 0, 0, 7, 0, 0],
    [2, 7, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 2, 0, 0, 0, 0, 2],
])

In [3]:
B = np.array([
    [8, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 0, 0, 8, 0, 0],
    [0, 0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [1, 0, 4, 0, 0, 2, 0, 0, 0],
    [1, 0, 2, 0, 0, 0, 1, 0, 0],
    [2, 9, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 2, 0, 0, 0, 0, 3],
])

In [4]:
count = 0
  
rows = len(A)
cols = len(A[0])
  
size = rows * cols
   
for i in range(0, rows):  
    for j in range(0, cols):  
        if(A[i][j] == 0):  
            count = count + 1

if(count > (size/2)):  
    print("Данная матрица разреженная")
else:  
    print("Данная матрица плотная")

Данная матрица разреженная


In [5]:
# калькуляция разреженности матрицы A
sparsity = 1.0 - np.count_nonzero(A) / A.size
print(sparsity)

0.7901234567901234


In [6]:
# функция упаковки матрицы
def get_csr_representation(matrix):
    rows_index = []
    columns = []
    elements = []
    row_len = len(matrix)
    col_len = len(matrix[0])
    for i in range(row_len):
        rows_index.append(len(columns))
        for j in range(col_len):
            if(matrix[i][j] != 0):
                columns.append(j)
                elements.append(matrix[i][j])
    rows_index.append(len(columns))
    
    return rows_index, columns, elements

In [7]:
# функция умножения упакованных матриц
def sparse_csr_multiplication(matrix_A, matrix_B):
    rows, columns, elements = matrix_A
    rows_count = len(matrix_B)
    col_count = len(matrix_B[0])
    C = [[0 for _ in range(col_count)] for _ in range(rows_count)]
    flop_count = 0
    for col in range(col_count):
        for row in range(rows_count):
            for j in range(rows[row],rows[row+1]):
                flop_count += 2
                C[row][col] += elements[j] * matrix_B[columns[j]][col]
                
    return C, flop_count

In [8]:
print('Исходная матрица A:')
print(A)

Исходная матрица A:
[[5 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 2 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0]
 [1 0 4 0 0 4 0 0 0]
 [1 0 3 0 0 0 7 0 0]
 [2 7 0 0 0 0 0 1 0]
 [0 0 0 2 0 0 0 0 2]]


In [9]:
print('Исходная матрица B:')
print(B)

Исходная матрица B:
[[8 0 0 0 0 0 0 0 0]
 [0 2 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 8 0 0]
 [0 0 0 3 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0]
 [1 0 4 0 0 2 0 0 0]
 [1 0 2 0 0 0 1 0 0]
 [2 9 0 0 0 0 0 1 0]
 [0 0 0 2 0 0 0 0 3]]


In [10]:
print('Упакованная CSR матрица A:')
A_csr = get_csr_representation(A)
A_csr

Упакованная CSR матрица A:


([0, 1, 2, 4, 5, 6, 9, 12, 15, 17],
 [0, 1, 0, 2, 3, 4, 0, 2, 5, 0, 2, 6, 0, 1, 7, 3, 8],
 [5, 1, 1, 2, 3, 4, 1, 4, 4, 1, 3, 7, 2, 7, 1, 2, 2])

In [11]:
print('Упакованная CSR матрица B:')
B_csr = get_csr_representation(B)
B_csr

Упакованная CSR матрица B:


([0, 1, 2, 5, 6, 7, 10, 13, 16, 18],
 [0, 1, 0, 2, 6, 3, 4, 0, 2, 5, 0, 2, 6, 0, 1, 7, 3, 8],
 [8, 2, 1, 1, 8, 3, 4, 1, 4, 2, 1, 2, 1, 2, 9, 1, 2, 3])

In [12]:
print('Распаковка:')

Распаковка:


In [13]:
print('Умножение A*A:')
print(sparse_csr_multiplication(A_csr, A_csr))

Умножение A*A:
([[0, 5, 10, 20, 25, 30, 45, 60, 75, 85], [0, 1, 0, 2, 3, 4, 0, 2, 5, 0], [10, 3, 4, 8, 11, 14, 11, 20, 23, 19]], 80)


In [14]:
print('Умножение A*B:')
print(sparse_csr_multiplication(A_csr, A_csr))

Умножение A*B:
([[0, 5, 10, 20, 25, 30, 45, 60, 75, 85], [0, 1, 0, 2, 3, 4, 0, 2, 5, 0], [10, 3, 4, 8, 11, 14, 11, 20, 23, 19]], 80)


In [15]:
print('Умножение:')
print(sparse_csr_multiplication(A_csr, B))

Умножение:
([[40, 0, 0, 0, 0, 0, 0, 0, 0], [0, 2, 0, 0, 0, 0, 0, 0, 0], [10, 0, 2, 0, 0, 0, 16, 0, 0], [0, 0, 0, 9, 0, 0, 0, 0, 0], [0, 0, 0, 0, 16, 0, 0, 0, 0], [16, 0, 20, 0, 0, 8, 32, 0, 0], [18, 0, 17, 0, 0, 0, 31, 0, 0], [18, 23, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 10, 0, 0, 0, 0, 6]], 306)


In [16]:
print('Сложение:')

Сложение:


In [17]:
get_csr_representation(A*A)

([0, 1, 2, 4, 5, 6, 9, 12, 15, 17],
 [0, 1, 0, 2, 3, 4, 0, 2, 5, 0, 2, 6, 0, 1, 7, 3, 8],
 [25, 1, 1, 4, 9, 16, 1, 16, 16, 1, 9, 49, 4, 49, 1, 4, 4])

## Проверка
Для проверки результатов используем метод [csr_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) (Compressed Sparse Row matrix) модуля [Sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html) из библиотеки [SciPy](https://docs.scipy.org/doc/scipy/reference/index.html)

In [19]:
# convert to sparse matrix (Compressed Sparse Row - CSR method)
print('Упакованная CSR матрица:')
S = csr_matrix(A)
print(S)

Распакованная матрица:
[[5 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [1 0 2 0 0 0 0 0 0]
 [0 0 0 3 0 0 0 0 0]
 [0 0 0 0 4 0 0 0 0]
 [1 0 4 0 0 4 0 0 0]
 [1 0 3 0 0 0 7 0 0]
 [2 7 0 0 0 0 0 1 0]
 [0 0 0 2 0 0 0 0 2]]
